##  DIT Natural Language Processing lesson 2025

# Long Short-Term Memory Networks

In [ ]:
! pip install gensim

In [ ]:
# Importing the dependencies
import glob
import numpy as np
import os
import tarfile

from gensim.models.keyedvectors import KeyedVectors

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, LSTM # <--

from nltk.tokenize import TreebankWordTokenizer
from random import shuffle
from tqdm.auto import tqdm
from urllib import request


In [ ]:
PATH_TO_CORPUS = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
CORPUS_FILE_NAME = "aclImdb_v1.tar.gz"

PATH_TO_GOOGLENEWS_VECTORS ="https://www.dropbox.com/s/965dir4dje0hfi4/GoogleNews-vectors-negative300.bin.gz?dl=1"
GOOGLE_VECTORS = "GoogleNews-vectors-negative300.bin.gz"

CORPUS_PATH = "aclImdb/train"

def download_file(url_to_file, path_to_file):
  if os.path.isfile(path_to_file):
    print("A local copy of the file exists already:", path_to_file, "\nDoing nothing")
  else:
    request.urlretrieve(url_to_file, path_to_file)

In [ ]:
# Downloading the embeddings
download_file(PATH_TO_GOOGLENEWS_VECTORS, GOOGLE_VECTORS)

# Downloading and untaring the corpus
download_file(PATH_TO_CORPUS, CORPUS_FILE_NAME)
with tarfile.open(CORPUS_FILE_NAME) as f:
  f.extractall(path=".")

In [ ]:
# Loading the embeddings
word_vectors = KeyedVectors.load_word2vec_format(GOOGLE_VECTORS,
    binary=True, limit=400000)

In [ ]:
# I am using the same preprocessing functions again

def pre_process_data(filepath):
    """
    Load pos and neg examples from separate dirs then shuffle them
    together.
    """
    positive_path = os.path.join(filepath, 'pos')
    negative_path = os.path.join(filepath, 'neg')
    pos_label = 1
    neg_label = 0
    dataset = []
    for filename in glob.glob(os.path.join(positive_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((pos_label, f.read()))
    for filename in glob.glob(os.path.join(negative_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((neg_label, f.read()))
    shuffle(dataset)
    return dataset

# Tokenizing and vectorizing all the instances
def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])
            except KeyError:
                pass
        vectorized_data.append(sample_vecs)
    return vectorized_data

def collect_expected(dataset):
    """Extracting the expected output for all the instances"""
    return [sample[0] for sample in dataset]

In [ ]:
# Data preparation
dataset = pre_process_data(CORPUS_PATH)
vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)
split_point = int(len(vectorized_data) * .8)

x_train = vectorized_data[:split_point]
y_train = expected[:split_point]

x_test = vectorized_data[split_point:]
y_test = expected[split_point:]

In [ ]:
dataset

In [ ]:
# Network parameters

maxlen = 400
embedding_dims = 300

batch_size = 32
epochs = 2
num_neurons = 50

In [ ]:
# Padding and truncating the input is not strictly necessary for RNNs; we
# due to the recursion strategy we are applying here and to have a fair
# comparison against the CNN (same input length)
class Collator:
  def __init__(self,
                maxlen,
                batch_size,
                ) -> None:
    self.maxlen = maxlen
    self.batch_size = batch_size

  def padding_and_truncating(self, x, y):
    """
    Add zeros at the end of the representation for short instances,
    truncate longer ones to the maxlen
    """
    vec_dim = len(x[0][0])
    N = len(x)
    X = np.zeros((N, self.maxlen, vec_dim))  # preallocate a np array
    Y = np.array(y)

    for i, tokens in enumerate(x):
      length = min(len(tokens), self.maxlen)
      if length > 0:
        X[i, :length] = np.asarray(tokens[:length])  # fill the np array
    return X, Y

  def collate(self, X, Y, N, epochs = 1):
    """
    This method is used to feed batches into the `model.fit` method
    """
    for _ in range(epochs):
      """
      This `for _ in range(epochs):` loop is here because
      the `for` below needs to be able to be called `epochs` times
      so each time the `for` is called the iterator is replenished
      for the new epoch
      """
      for start in range(0, N, self.batch_size):
        end = start + self.batch_size
        x_batch, y_batch = self.padding_and_truncating(
          X[start:end],
          Y[start:end]
          )
        yield x_batch, y_batch

In [ ]:
collator = Collator(maxlen = maxlen, batch_size = batch_size)

In [ ]:
model = Sequential()

model.add(Input([maxlen, embedding_dims]))
model.add(
    LSTM(num_neurons,
         return_sequences=True,
        )
    )

# This is what we had for the biRNN
# model.add(Bidirectional(SimpleRNN(
#     num_neurons,
#     return_sequences=True),
#     )
#  )

In [ ]:
# Adding a dropout, flattening, and classification layers
model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compiling the network
model.compile('rmsprop',
              'binary_crossentropy',
              metrics=['accuracy'])
print(model.summary())

That's **70,200** parameters in the LSTM (against **17,550** for the RNN)

Back to the slides

In [ ]:
# Training the network

model.fit(
    collator.collate(x_train, y_train, N = len(x_train), epochs=epochs),
    steps_per_epoch=len(x_train) // batch_size,
    validation_data=collator.collate(x_test, y_test, N = len(x_test), epochs=epochs),
    validation_steps=len(x_test) // batch_size,
    epochs=epochs
    )


In [ ]:
# Saving the network for future use
model_structure = model.to_json()
with open("lstm_model1.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights("lstm.weights.h5")

In [ ]:
# Notice we have both positive and negative words here
sample_negative = """I hate that the dismal weather had me down for so long,
when will it break! Ugh, when does happiness return? The sun is
blinding and the puffy clouds are too thin. I can't wait for the weekend."""
# Super positive sample
sample_positive = """I love that incredible weather!
I feel like this happiness will last forever!
The sun is super nice and the breeze is soothing.
I could stay like this forever.
"""

# The first value is a "fake" class (this is the expected input)
data_dummy = [(0, sample_negative), (0, sample_positive)]
x_dummy = tokenize_and_vectorize(data_dummy)
y_dummy = collect_expected(data_dummy)


# model.predict(test_vec)
x_dummy_padded, y_dummy = collator.padding_and_truncating(x_dummy, y_dummy)
preds = model.predict(x_dummy_padded)
print(preds)

In [ ]:
# Get the class by thresholding the logits
(preds > 0.5).astype("int32")

**End of the notebook**